# Fatorações e outras brincadeiras
Baseado no trabalho de Andreas Noack

## Resumo
 - Fatorações
 - Matrizes com estruturas especiais
 - Álgebra linear genérica

Antes de começar, vamos criar um sistema linear e usar `LinearAlgebra` para carregar as fatorizações and matrizes com estrutura special.

In [ ]:
using LinearAlgebra
A = rand(3, 3)
x = fill(1, (3,))
b = A * x

## Fatorações

#### Fatoração ou decomposição LU 
Em Julia podemos fazer a decomposição LU 
```julia
PA = LU
``` 
onde `P` é a matriz de permutação, `L` é a matriz triangular inferior com diagonal unitária e `U` é a matriz superior triangular, usando `lufact`.

Julia permite o cálculo da decomposição LU e define um tipo de decomposição para armazená-la.

In [ ]:
Alu = lu(A)

In [ ]:
typeof(Alu)

As diferentes partes da fatoração podem ser obtidas acessando suas propriedades especiais

In [ ]:
Alu.P

In [ ]:
Alu.L

In [ ]:
Alu.U

Julia pode despachar métodos em objetos de fatoração.

Como exemplo, podemos resolver sistemas lineares utilizando a matriz original ou o objeto de fatoração.

In [ ]:
A\b

In [ ]:
Alu\b

De maneira análoga, podemos calcular a determinante de  `A` usando  `A` ou o objeto de fatoração

In [ ]:
det(A) ≈ det(Alu)

#### Fatoração QR

Em Julia podemos realizar a decomposição QR
```
A=QR
``` 

onde  `Q` é unitária/ortogonal e `R` é uma matriz triangular superior, usando `qrfact`. 

In [ ]:
Aqr = qr(A)

Igualmente à decomposição LU,  as matrizes `Q` e `R` podem ser extraídas da decomposição QR  via

In [ ]:
Aqr.Q

In [ ]:
Aqr.R

#### Decomposição em auto-vetores

Os resultados de decomposição em auto-vetores, decomposição de valor singular, fatoração de Hessemberg e decomposição de Schur são armazenados nos tipos `Factorization`.

A decomposição em auto-vetores pode ser calculada 

In [ ]:
Asym = A + A'
AsymEig = eigen(Asym)

Os valores e os vetores podem ser extraídos utilizando indexação especial

In [ ]:
AsymEig.values

In [ ]:
AsymEig.vectors

Novamente, quando a fatoração é armazenada em um tipo específico, podemos despachar neste tipo e escrever métodos especializados que exploram as propriedades da fatoração, isto é, que $A^{-1}=(V\Lambda V^{-1})^{-1}=V\Lambda^{-1}V^{-1}$.

In [ ]:
inv(AsymEig)*Asym

## Matrizes com estrutura especial
A estrutura de uma matriz é muito importante em álgebra linear. Para ver o *quão* importante é, vamos trabalhar com um sistema linear grande

In [ ]:
n = 1000
A = randn(n,n);

Julia pode algumas vezes inferir a estrutura da matriz especial

In [ ]:
Asym = A + A'
issymmetric(Asym)

mas algumas vezes erros de ponto flutuante podem atrapalhar de maneira considerável.

In [ ]:
Asym_noisy = copy(Asym)
Asym_noisy[1,2] += 5eps()

In [ ]:
issymmetric(Asym_noisy)

Por sorte, podemos declarar explicitamente a estrutura da matriz, como por exemplo, `Diagonal`, `Triangular`, `Symmetric`, `Hermitian`, `Tridiagonal` e `SymTridiagonal`.

In [ ]:
Asym_explicit = Symmetric(Asym_noisy);

Vamos comparar o tempo que leva para Julia calcular os auto-valores de  `Asym`, `Asym_noisy`, e `Asym_explicit`

In [ ]:
@time eigvals(Asym);

In [ ]:
@time eigvals(Asym_noisy);

In [ ]:
@time eigvals(Asym_explicit);

Neste example, usando `Symmetric()` em `Asym_noisy` fez os nossos cálculos aproximadamente about `5x` mais efficiente :)

#### Um problema grande
Usando os tipos  `Tridiagonal` e `SymTridiagonal` types para armazenar matrizes tridiagonais é possível trabalhar com problemas tridiagonais enormes. O exemplo a seguir não seria possível neste laptop se a matriz fosse armazenada como uma matriz densa de tipo `Matrix`.

In [ ]:
n = 1_000_000;
A = SymTridiagonal(randn(n), randn(n-1));
@time eigmax(A)

## Álgebra linear genérica
O jeito usual de adicionar suporte a álgebra linear numérica é fazer uma interface às funções disponíveis nas bibliotecas BLAS e LAPACK. Para matrizes como elementos com tipos `Float32`, `Float64`, `Complex{Float32}` ou `Complex{Float64}` é exatamente isso que Julia faz (assim como numpy/python, matlab, R, etc).

No entanto, Julia também implementa álgebra linear genérica, permitindo que você trabalhe com matrizes de outros tipos como racionais por exemplo.

#### Números racionais
Julia já vem com números racionais. Para construir números racionais, use a barra pra frente dupla:

In [ ]:
1//2

#### Exemplo: Sistema de equações linear racional
O exemplo a seguir mostra como um sistema de equações lineares com elementos racionais pode ser resolvido sem utilizar ponto flutuante. Cuidado que à medida que estas matrizes crescem, estouro (_overflow_) é um problema comum e eventualmente Inteiros grandes (`BigInt`) são necessários.

In [ ]:
Arational = Matrix{Rational{BigInt}}(rand(1:10, 3, 3))/10

In [ ]:
x = fill(1, 3)
b = Arational*x

In [ ]:
Arational\b

In [ ]:
lu(Arational)

### Exercícios

#### 11.1
Quais são os auto-valores da matriz A?

```
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]
```

#### 11.2 
Create a `Diagonal` matrix from the eigenvalues of `A`.

#### 11.3 
Create a `LowerTriangular` matrix from `A`.

### Please let us know how we're doing!
https://tinyurl.com/introJuliaFeedback